In [ ]:
%pip install pandas
import pandas as pd

In [ ]:
prop_data = pd.read_csv('../data/all_sold_property_data.csv')

In [6]:
prop_data.columns

Index(['index', 'address', 'city', 'state', 'zipcode', 'county', 'countyFIPS',
       'latitude', 'longitude', 'bathrooms', 'bathroomsFull', 'bathroomsHalf',
       'bedrooms', 'homeStatus', 'homeType', 'livingArea', 'lotSize', 'price',
       'currency', 'description', 'contingentListingType', 'datePostedString',
       'datePriceChanged', 'dateSoldString', 'daysOnZillow', 'favoriteCount',
       'isListedByOwner', 'isNonOwnerOccupied', 'isPreforeclosureAuction',
       'isBankOwned', 'isForeclosure', 'isFSBA', 'isFSBO', 'isComingSoon',
       'isForAuction', 'isNewHome', 'isOpenHouse', 'isPending',
       'isZillowOwned', 'keystoneHomeStatus', 'lastSoldPrice',
       'listingTypeDimension', 'listing_agent', 'livingAreaUnits',
       'livingAreaUnitsShort', 'livingAreaValue', 'lotAreaUnits',
       'lotAreaValue', 'taxAssessedValue', 'taxAssessedYear', 'taxHistory',
       'appliances', 'architecturalStyle', 'heating', 'cooling', 'fencing',
       'flooring', 'basement', 'yearBuilt', 

####Data Cleaning:
retain only the most important attributes 

In [27]:
stripped_data = prop_data[['address', 'bathrooms', 'bedrooms', 'homeType', 'livingArea', 'lotSize', 'taxAssessedValue', 'yearBuilt', 'price']]
stripped_data.set_index('address')
stripped_data = stripped_data.dropna()
stripped_data.size

63630

In [ ]:
%pip install -U scikit-learn

####Split dataset into training and testing sets 

In [37]:
from sklearn.model_selection import train_test_split
prop_val = stripped_data[stripped_data.columns[:-1]]
prop_price = stripped_data[['address', 'price']]
val_train, val_test, price_train, price_test = train_test_split(prop_val, prop_price, 
                                                                test_size=0.3, random_state=1)